# Localization_Classification.ipynb
Date: November 29th, 2018  
Course: ECSE415, McGill University  
Authors:  
*Shawn Vosburg  
Tristan Bouchard  
Alex Masciotra  
Nayem Alam  
Thomas Philippon *

## 3. Localization & Classification

In [79]:
#Import the required libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import  random as rand
import csv


In [101]:
#Class hoding the dimensions of the object windows.  
class rectangle():
    def __init__(self, x, y, w, h,color = (0,0,0)):
        self.x = x
        self.y = y
        self.w = w
        self.h = h
        self.color = color
    def print(self):
        print(self.x,self.y,self.w,self.h)
        
#Image directory 
folder = "./MIO-TCD-Localization/Localization/train/"
csv_datapath = "./MIO-TCD-Localization/MIO-TCD-Localization/gt_train.csv"

#function reading a .csv file and creating a rectangle object from the GT table
def get_gt(img_id, csv_datapath):
    
    """Function that takes an image from the dataset and the GT csv file and returns an array of rectangles each detected object on the image 
    ----------
    img_id : image name (name of the file) 
    csv_file : csv file read using the csv.read function
    Returns 
    -------
    rectangle_list : list of rectangle objects for each detected object on the image 
    """
    #create the list of rectangles object 
    rectangle_list = []
    
    #open the csv file and retrieve the gt
    with open(csv_datapath) as f: 
        reader = csv.reader(f)
        for i in reader:
            if(i[0] == img_id): 
                rect = rectangle(x = i[2], y = i[4] , h = i[3] ,w = i[5])
                rectangle_list.append(rect)
    
    return rectangle_list 


#function to compute the dice coefficients 
def dice_coef(gt_list, img_shape, img_tag, csv_datapath):
    
    """Function that takes the detected windows from the localization algorithm and computes the dice coefficients using the gt table
       The groung truth rectangles are retrived from the .csv file 
    ----------
    csv_file : csv file read using the csv.read function
    img_shape : shape of the image (img.shape)
    img_tag : image name in the data set 
    csv_datapath : datapath to the folder where the csv file is 
    
    Returns 
    -------
    dice_list : dice coefficient for each ground truth image 
    """
    #create the list of ground truth rectangles object 
    pred_list = []
    
    #open the csv file and retrieve the gt
    with open(csv_datapath) as f: 
        reader = csv.reader(f)
        for i in reader:
            if(i[0] == img_tag): 
                rect = rectangle(x = i[2], y = i[4] , h = i[3] ,w = i[5])
                pred_list.append(rect)

    #now that the ground truth recangles have been retreived from the .csv file, compute the dce coeff. 
    #list of Dice coefficient for the detected rectangles
    dice_list = [] 

    for gt in range(len(gt_list)): 
        gt_img = np.zeros(img_shape)
        dice_max = []
        
        #retrieve the ground truth rectangle parameters
        gt_x = int(gt_list[gt].x)
        gt_y = int(gt_list[gt].y)
        gt_w = int(gt_list[gt].w)
        gt_h = int(gt_list[gt].h)
        
        #draw the filled rectangle for the ground truth rectangle  
        cv2.rectangle(gt_img, (gt_x, gt_y), (gt_x + gt_w -1, gt_y + gt_h -1), 1.0, -1)
 

        for pred in range(len(pred_list)): 
            TP_img = np.zeros(img_shape)
            FP_img = np.zeros(img_shape)
            FN_img = np.zeros(img_shape)
            flat = np.zeros([0,img_shape[0]*img_shape[1]])
            pred_img = np.zeros(img_shape)
            
            #retrieve the predicted rectangle parameters
            pred_x = int(pred_list[pred].x)
            pred_y = int(pred_list[pred].y)
            pred_w = int(pred_list[pred].w)
            pred_h = int(pred_list[pred].h)
            
            #draw the filled rectangle for the predicted rectangle 
            cv2.rectangle(pred_img, (pred_x, pred_y), ((pred_x + pred_w -1), (pred_y + pred_h -1)), (1.0, 1.0, 1.0), -1)  
            
            #TP is the multiplication of the two images, since the rectangle region is 1 and the rest 0 (like a bitwise AND for each pixel )
            TP_img = gt_img * pred_img
            TP = sum(TP_img.flatten())
            
            #FP is the predicted image minus TP 
            FP_img = pred_img - TP_img
            FP = sum(FP_img.flatten())
            
            #FN is the predicted image minus TP 
            FN_img = gt_img - TP_img
            FN = sum(FN_img.flatten())
            
            #compute the DICE coefficient 
            dice_max.append((2*TP)/(2*TP+FN+FP))
            
            
        dice_list.append(max(dice_max))
            
    return dice_list 


a = read_image("00000000", csv_datapath)

d = dice_coef(a, [800, 800], "00000001", csv_datapath );
print(d)

[0.0, 0.0, 0.0, 0.0, 0.0]
